In [1]:
from datasets import load_dataset, Dataset
import numpy as np

In [14]:
from dialogue2graph.pipelines.core.dialogue import Dialogue
from dialogue2graph.pipelines.core.graph import Graph

In [2]:
dataset = load_dataset("DeepPavlov/d2g_generated_augmented", token=True)
dataset

DatasetDict({
    train: Dataset({
        features: ['graph', 'topic', 'dialogues', 'augmented_dialogues'],
        num_rows: 376
    })
})

# Part 1

- |{dialog+aug_dialog}|= N ~ G1

- |{paths}|= N ~ G1


G1


p1 > {selected_p}

{paths - 1} -> p2 - p1 = 4

{paths - 1} -> p3 - p1 = 2

{paths - 1} -> p4 - p1 = 10 <<<(max_new_elements)

{paths - 1} -> p5 - p1 = 1


p4 > {selected_p}

selected_p = p1 + p4

{paths - 2} -> p2 - (p1 + p4) = 0

{paths - 2} -> p3 - (p1 + p4) = 1 <<<(max_new_elements)

{paths - 2} -> p5 - (p1 + p4) = 0


p3 > {selected_p}

selected_p = p1 + p4 + p3

{paths - 2} -> p2 - (p1 + p4 +p3) = 0

{paths - 2} -> p5 - (p1 + p4 +p3) = 0

## Functions

In [209]:
def get_start_dialogue(augmented_dialogues):
    max_len = 0
    idx_to_choose_from = []
    for aug_dia in augmented_dialogues:
        if len(aug_dia['messages']) >= max_len:
            max_len = len(aug_dia['messages'])

    for i, aug_dia in enumerate(augmented_dialogues):
        if len(aug_dia['messages']) == max_len:
            idx_to_choose_from.append(i)

    if len(idx_to_choose_from) == 1:
        idx = idx_to_choose_from[0]
    else:
        idx = np.random.choice(idx_to_choose_from)
    
    return idx, augmented_dialogues[idx]

In [211]:
def add_elements_to_paths(start_dialogue, graph):
    # add id to each edge
    for edge in graph['edges']:
        edge_id = str(edge['source']) + '_' + str(edge['target'])
        edge['id'] = edge_id

    paths = {}
    paths['nodes'] = []; paths['edges'] = []

    for turn in start_dialogue['messages']:
        if turn['participant'] == 'assistant':
            key = 'nodes'
        elif turn['participant'] == 'user':
            key = 'edges'

        for element in graph[key]:
            if turn['text'] in element['utterances']:
                paths[key].append(element['id'])
        
    paths['nodes'] = set(paths['nodes'])
    paths['edges'] = set(paths['edges'])
                
    return paths

In [212]:
def count_new_elements(paths, new_paths):
    new_nodes = new_paths['nodes'].difference(paths['nodes'])
    new_edges = new_paths['edges'].difference(paths['edges'])
    new_elements = len(new_nodes) + len(new_edges)
    return new_elements

In [213]:
def get_max_new_elements(augmented_dialogues, added_idx, graph, paths):
    all_new_elements_per_dia = []
    all_paths_per_dia = []
    max_new_elements = 0

    for i, aug_dia in enumerate(augmented_dialogues):
        if i in added_idx:
            continue
        else:
            new_paths = add_elements_to_paths(aug_dia, graph)
            all_paths_per_dia.append((i, new_paths))

            new_elements = count_new_elements(paths, new_paths)        
            all_new_elements_per_dia.append((i, new_elements)) 

            if new_elements >= max_new_elements:
                max_new_elements = new_elements
    
    return all_new_elements_per_dia, all_paths_per_dia, max_new_elements

In [214]:
def get_idx_and_paths_to_add(added_idx, all_new_elements_per_dia, all_paths_per_dia, max_new_elements):
    if max_new_elements == 0:
        return (None, f'no new elements in remaining dialogues')
    
    idx_to_choose_from = []
        
    for i, count in all_new_elements_per_dia:
        if i in added_idx:
            continue
        else:
            if count == max_new_elements:
                idx_to_choose_from.append(i)
    
    if len(idx_to_choose_from) == 1:
        idx_to_add = idx_to_choose_from[0]
    else:
        idx_to_add = np.random.choice(idx_to_choose_from)

    for i, new_paths in all_paths_per_dia:
        if i == idx_to_add:
            paths_to_add = new_paths
                
    return idx_to_add, paths_to_add

In [215]:
def add_new_elements_to_paths(paths, new_elements):
    paths['nodes'] = paths['nodes'].union(new_elements['nodes'])
    paths['edges'] = paths['edges'].union(new_elements['edges'])
    return paths

## Testing

In [250]:
example = dataset['train'][0]
graph = example['graph']
augmented_dialogues = example['dialogues']
len(augmented_dialogues)

6

In [251]:
start_idx, start_dia = get_start_dialogue(augmented_dialogues)
start_idx, len(start_dia['messages'])

(1, 23)

In [252]:
paths = add_elements_to_paths(start_dia, graph)
len(paths['nodes']), len(paths['edges'])

(8, 8)

In [253]:
paths

{'nodes': {1, 2, 4, 5, 6, 7, 8, 9},
 'edges': {'1_2', '2_5', '5_6', '6_7', '7_8', '8_4', '8_9', '9_5'}}

round 1

In [254]:
added_idx = []
added_idx.append(start_idx)

all_new_elements_per_dia, all_paths_per_dia, max_new_elements = get_max_new_elements(augmented_dialogues, added_idx, graph, paths)
all_new_elements_per_dia, all_paths_per_dia, max_new_elements

([(0, 0), (2, 1), (3, 2), (4, 2), (5, 1)],
 [(0,
   {'nodes': {1, 2, 4, 5, 6, 7, 8, 9},
    'edges': {'1_2', '2_5', '5_6', '6_7', '7_8', '8_4', '8_9', '9_5'}}),
  (2,
   {'nodes': {1, 2, 4, 5, 6, 7, 8},
    'edges': {'1_2', '2_5', '5_6', '6_2', '6_7', '7_8', '8_4'}}),
  (3, {'nodes': {1, 3}, 'edges': {'1_3'}}),
  (4, {'nodes': {1, 3}, 'edges': {'1_3'}}),
  (5, {'nodes': {1, 4}, 'edges': {'1_4'}})],
 2)

In [255]:
idx_to_add, paths_to_add = get_idx_and_paths_to_add(added_idx, all_new_elements_per_dia, all_paths_per_dia, max_new_elements)
idx_to_add, paths_to_add

(3, {'nodes': {1, 3}, 'edges': {'1_3'}})

In [256]:
added_idx.append(idx_to_add)
paths = add_new_elements_to_paths(paths, paths_to_add)
added_idx, paths

([1, 3],
 {'nodes': {1, 2, 3, 4, 5, 6, 7, 8, 9},
  'edges': {'1_2', '1_3', '2_5', '5_6', '6_7', '7_8', '8_4', '8_9', '9_5'}})

2 round

In [257]:
all_new_elements_per_dia, all_paths_per_dia, max_new_elements = get_max_new_elements(augmented_dialogues, added_idx, graph, paths)
all_new_elements_per_dia, all_paths_per_dia, max_new_elements

([(0, 0), (2, 1), (4, 0), (5, 1)],
 [(0,
   {'nodes': {1, 2, 4, 5, 6, 7, 8, 9},
    'edges': {'1_2', '2_5', '5_6', '6_7', '7_8', '8_4', '8_9', '9_5'}}),
  (2,
   {'nodes': {1, 2, 4, 5, 6, 7, 8},
    'edges': {'1_2', '2_5', '5_6', '6_2', '6_7', '7_8', '8_4'}}),
  (4, {'nodes': {1, 3}, 'edges': {'1_3'}}),
  (5, {'nodes': {1, 4}, 'edges': {'1_4'}})],
 1)

In [258]:
idx_to_add, paths_to_add = get_idx_and_paths_to_add(added_idx, all_new_elements_per_dia, all_paths_per_dia, max_new_elements)
idx_to_add, paths_to_add

(5, {'nodes': {1, 4}, 'edges': {'1_4'}})

In [259]:
added_idx.append(idx_to_add)
paths = add_new_elements_to_paths(paths, paths_to_add)
added_idx, paths

([1, 3, 5],
 {'nodes': {1, 2, 3, 4, 5, 6, 7, 8, 9},
  'edges': {'1_2',
   '1_3',
   '1_4',
   '2_5',
   '5_6',
   '6_7',
   '7_8',
   '8_4',
   '8_9',
   '9_5'}})

round 3

In [260]:
all_new_elements_per_dia, all_paths_per_dia, max_new_elements = get_max_new_elements(augmented_dialogues, added_idx, graph, paths)
all_new_elements_per_dia, all_paths_per_dia, max_new_elements

([(0, 0), (2, 1), (4, 0)],
 [(0,
   {'nodes': {1, 2, 4, 5, 6, 7, 8, 9},
    'edges': {'1_2', '2_5', '5_6', '6_7', '7_8', '8_4', '8_9', '9_5'}}),
  (2,
   {'nodes': {1, 2, 4, 5, 6, 7, 8},
    'edges': {'1_2', '2_5', '5_6', '6_2', '6_7', '7_8', '8_4'}}),
  (4, {'nodes': {1, 3}, 'edges': {'1_3'}})],
 1)

In [261]:
idx_to_add, paths_to_add = get_idx_and_paths_to_add(added_idx, all_new_elements_per_dia, all_paths_per_dia, max_new_elements)
idx_to_add, paths_to_add

(2,
 {'nodes': {1, 2, 4, 5, 6, 7, 8},
  'edges': {'1_2', '2_5', '5_6', '6_2', '6_7', '7_8', '8_4'}})

In [262]:
added_idx.append(idx_to_add)
paths = add_new_elements_to_paths(paths, paths_to_add)
added_idx, paths

([1, 3, 5, 2],
 {'nodes': {1, 2, 3, 4, 5, 6, 7, 8, 9},
  'edges': {'1_2',
   '1_3',
   '1_4',
   '2_5',
   '5_6',
   '6_2',
   '6_7',
   '7_8',
   '8_4',
   '8_9',
   '9_5'}})

round 4

In [263]:
all_new_elements_per_dia, all_paths_per_dia, max_new_elements = get_max_new_elements(augmented_dialogues, added_idx, graph, paths)
all_new_elements_per_dia, all_paths_per_dia, max_new_elements

([(0, 0), (4, 0)],
 [(0,
   {'nodes': {1, 2, 4, 5, 6, 7, 8, 9},
    'edges': {'1_2', '2_5', '5_6', '6_7', '7_8', '8_4', '8_9', '9_5'}}),
  (4, {'nodes': {1, 3}, 'edges': {'1_3'}})],
 0)

In [264]:
idx_to_add, paths_to_add = get_idx_and_paths_to_add(added_idx, all_new_elements_per_dia, all_paths_per_dia, max_new_elements)
idx_to_add, paths_to_add

(None, 'no new elements in remaining dialogues')

### whole algorithm:

In [269]:
start_idx, start_dia = get_start_dialogue(augmented_dialogues)
paths = add_elements_to_paths(start_dia, graph)
print(paths)

added_idx = []
added_idx.append(start_idx)
iters = 0
while iters < 100:
    all_new_elements_per_dia, all_paths_per_dia, max_new_elements = get_max_new_elements(
        augmented_dialogues, added_idx, graph, paths
        )
    idx_to_add, paths_to_add = get_idx_and_paths_to_add(
        added_idx, all_new_elements_per_dia, all_paths_per_dia, max_new_elements
        )
    if paths_to_add == 'no new elements in remaining dialogues':
        break
    else:
        added_idx.append(idx_to_add)
        paths = add_new_elements_to_paths(paths, paths_to_add)
added_idx, paths

{'nodes': {1, 2, 4, 5, 6, 7, 8, 9}, 'edges': {'7_8', '5_6', '2_5', '6_7', '8_9', '9_5', '1_2', '8_4'}}


([0, 3, 5, 2],
 {'nodes': {1, 2, 3, 4, 5, 6, 7, 8, 9},
  'edges': {'1_2',
   '1_3',
   '1_4',
   '2_5',
   '5_6',
   '6_2',
   '6_7',
   '7_8',
   '8_4',
   '8_9',
   '9_5'}})

It means we need only 4 dialogues out of 6 (ids 0, 2, 4, 5) to fully cover the graph.

# Part 2

|p1| = 3

|p4| = 1

|p3| = 1 <<< (min_dialog)


p3-d1 + p4-d1 union


p3 + p4 union p1 = share_elements


final_scores

for dialog in p1

sim_scores = []

for el in share_elements

sim_scores += [max(sim(dialog[el], p3-d1[el]),sim(dialog[el], p4-d1[el]))]


final_scores += sum(sim_scores)


minarg(final_scores) -> dialog_index

p1[dialog_index]